# **MACHINE LEARNING PROJECT EXAMS**

### Name : Amanda Ofori
### Course:  BSc. Computer Science
### Roll Number: 10201100146


## Question 2: **News Categorizing System**



##**Importing Packages**

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import requests
import re
import os
import pickle
from collections import Counter

## **Setting up Directory to store models**

In [ ]:
# Specify the directory name where you want to store the models
models_dir = "models"

# Create the directory if it does not exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Created directory: {models_dir}")
else:
    print(f"Directory already exists: {models_dir}")


Created directory: models


## **Loading the Dataset and Defining EDA and Preprocessing functions**

In [ ]:
# Load the data set
data = pd.read_csv('news-article-categories.csv', encoding='latin-1')

print("EDA on News Category dataset")
print(data.head())
print("**"*15)
# Get a concise summary of the dataframe
print(data.info())
print("**"*15)
# Summary statistics for numeric columns
print(data.describe())
print("**"*15)
# Count the number of missing values in each column
missing_values = data.isnull().sum()
print(missing_values)


# Fill NaN values in the 'body' column with empty strings
data['body'] = data['body'].fillna('')
print(data.columns)

# Preprocessing of thr dataset

 # Ensure you've downloaded necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # For lemmatization

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define preprocessing functions
def lemmatize_words(text):
    wordnet = WordNetLemmatizer()
    words = word_tokenize(text)
    return " ".join([wordnet.lemmatize(word) for word in words])

def remove_urls(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    return text

def remove_single_characters_and_excessive_spaces(text):
    # Remove single characters and excessive spaces
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove single characters surrounded by spaces
    text = re.sub(r'^[a-zA-Z]\s+', ' ', text)    # Remove single character at the start followed by a space
    text = re.sub(r'\s+', ' ', text, flags=re.I) # Remove excessive spaces
    return text.strip()  # Remove leading and trailing spaces

def convert_lower(text):
    return text.lower()

def remove_special_chars(text):
    text_without_special_chars = []
    for char in text:
        if char.isalnum():
            text_without_special_chars.append(char)
        else:
            text_without_special_chars.append(' ')
    return "".join(text_without_special_chars)

def remove_tags(text):
    tag_regex = re.compile(r'<[^>]+>')
    return re.sub(tag_regex, '', text)

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return " ".join([word for word in words if word not in stop_words])

def mode(results):
    freqs = Counter(results)
    ranking = sorted(freqs.items(), key=lambda pair: pair[1], reverse=True)
    return ranking[0][0]  # This returns the most frequent category name directly


# Define preprocessing functions
def preprocess_article_text(article_text):
    article_text = remove_tags(article_text)
    article_text = remove_special_chars(article_text)
    article_text = convert_lower(article_text)
    article_text = remove_stopwords(article_text)
    article_text = lemmatize_words(article_text)
    return article_text


# pre-process the text from the dataset
data['body'] = data['body'].apply(preprocess_article_text)
print(data.columns)



EDA on News Category dataset
         category                                              title  \
0  ARTS & CULTURE  Modeling Agencies Enabled Sexual Predators For...   
1  ARTS & CULTURE  Actor Jeff Hiller Talks âBright Colors And B...   
2  ARTS & CULTURE  New Yorker Cover Puts Trump 'In The Hole' Afte...   
3  ARTS & CULTURE  Man Surprises Girlfriend By Drawing Them In Di...   
4  ARTS & CULTURE  This Artist Gives Renaissance-Style Sculptures...   

                                                body  
0  In October 2017, Carolyn Kramer received a dis...  
1  This week I talked with actor Jeff Hiller abou...  
2  The New Yorker is taking on President Donald T...  
3  Kellen Hickey, a 26-year-old who lives in Huds...  
4  Thereâs something about combining the tradit...  
******************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6877 entries, 0 to 6876
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  --

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Index(['category', 'title', 'body'], dtype='object')


## **Defining the train models function**

In [ ]:
# Define the model
def train_models():
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data['body'], data['category'], test_size=0.3, random_state=42)

    # Initialize TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the training data and transform it
    X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)

    # Transform the test data using the same vectorizer
    X_test_vectorized = tfidf_vectorizer.transform(X_test)

    # Save model
    model_filename = os.path.join(models_dir, "vectoriser.pkl")
    if not os.path.exists(model_filename):
        with open(model_filename, 'wb') as f:
            pickle.dump(tfidf_vectorizer, f)
        print(f"Vectorizer saved to: {model_filename}")
    else:
        print(f"Vectorizer file already exists: {model_filename}")


    # Define individual models
    mnb = MultinomialNB()
    rfc = RandomForestClassifier(n_estimators=50, random_state=2)
    xgb = XGBClassifier(n_estimators=50, random_state=2)

    # Create a Voting Classifier
    voting_clf = VotingClassifier(
        estimators=[('mnb', mnb), ('rf', rfc), ('xgb', xgb)],
        voting='soft'
    )

    # Training the Ensemble Model
    voting_clf.fit(X_train_vectorized, y_train)

    # Hyperparameter Tuning for Random Forest
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }

    grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train_vectorized, y_train)
    rfc_best = grid_search.best_estimator_

    # Update the Voting Classifier with the best Random Forest model
    voting_clf.set_params(rf=rfc_best)

    # Re-train the updated Ensemble Model
    voting_clf.fit(X_train_vectorized, y_train)

    # Save the trained voting classifier to a file named 'voting_classifier.pkl'
    model_filename = os.path.join(models_dir, "voting_classifier.pkl")
    if not os.path.exists(model_filename):
        with open(model_filename, 'wb') as f:
            pickle.dump(voting_clf, f)
            print(f"Saved trained voting classifier to: {model_filename}")
    else:
        print(f"File already exists: {model_filename}. Skipping save operation.")

    # Model evaluation
    y_pred = voting_clf.predict(X_test_vectorized)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    return voting_clf, accuracy


## **Loading the Models for Prediction**

In [ ]:
# Load models and vectorizer for prediction
def load_models_and_vectorizer(models_path):
    vectorizer = None
    models = {}
    for root, _, files in os.walk(models_path):
        for file in files:
            model_path = os.path.join(root, file)
            with open(model_path, 'rb') as f:
                if file == 'vectoriser.pkl':
                    vectorizer = pickle.load(f)
                else:
                    model_name = file.split('.')[0]  # Extract model name from file name
                    models[model_name] = pickle.load(f)
    return vectorizer, models

## **Defining the Make_Prediction function**

In [ ]:
# Make prediction
def make_prediction(article_text):
    categories = [
        "ARTS & CULTURE",
        "BUSINESS",
        "COMEDY",
        "CRIME",
        "EDUCATION",
        "ENTERTAINMENT",
        "ENVIRONMENT",
    ]
    results = []
    absolute_path = os.path.dirname(__file__)
    models_path = os.path.join(absolute_path, 'models')
    vectorizer, trained_models = load_models_and_vectorizer(models_path)

    # Preprocess and vectorize the article text
    processed_article_text = preprocess_article_text(article_text)
    text_vectorized = vectorizer.transform([processed_article_text])

    # Make predictions using the loaded models
    for model_name, model in trained_models.items():
        prediction = model.predict(text_vectorized)[0]
        results.append(prediction)

    # Determine the most common prediction
    predicted_category = mode(results)
    return predicted_category  # Return the predicted category directly
